In [63]:
import pandas as pd
import numpy as np
df = pd.read_csv('output.csv')

In [66]:
df.head()

,date_time,device_id,tmp,hum,CO2,VOC,vis,IR,WIFI,BLE,...,spreading_factor,bandwidth,f_cnt,gateway_drag-lps8-01,gateway_drag-lps8-02,gateway_drag-lps8-03,gateway_drag-lps8-05,gateway_drag-lps8-07,gateway_drag-lps8-08,gateway_drag-outd-01
0,2022-10-10 14:09:26,hka-aqm-a014,21.10,55.26,528,450,488,61,2,3,...,7,125000,2,1,0,0,0,0,0,0
1,2022-10-10 14:18:26,hka-aqm-a014,22.37,51.95,526,450,528,72,2,2,...,7,125000,3,1,0,0,0,0,0,0
2,2022-10-10 14:18:47,hka-aqm-a014,22.44,51.73,525,450,647,102,2,1,...,7,125000,4,1,0,0,0,0,0,0
3,2022-10-10 14:19:07,hka-aqm-a014,22.50,51.63,525,450,766,130,2,1,...,7,125000,5,1,0,0,0,0,0,0
4,2022-10-10 14:19:26,hka-aqm-a014,22.56,51.41,526,450,877,157,1,1,...,7,125000,6,1,0,0,0,0,0,0


In [64]:
# changing date_time to datetime
df['date_time'] = pd.to_datetime(df['date_time'])

In [65]:
#onehot encoding the gateway column and changing its type to int
df = pd.get_dummies(df, columns=['gateway'])
for col in ['gateway_drag-lps8-01', 'gateway_drag-lps8-02', 'gateway_drag-lps8-03', 'gateway_drag-lps8-05', 'gateway_drag-lps8-07', 'gateway_drag-lps8-08', 'gateway_drag-outd-01']:
    df[col] = df[col].astype(int)

In [72]:
#splitting our columns into float and int types
all_columns_float_type = df.select_dtypes(include=['float64']).columns
all_columns_float_type = all_columns_float_type.tolist()
all_columns_float_type.append('device_id')
all_columns_float_type.append('date_time')
all_columns_int_type = df.select_dtypes(include=['int']).columns   
all_columns_int_type = all_columns_int_type.tolist()
all_columns_int_type.append('device_id')
all_columns_int_type.append('date_time')

In [73]:
# grouping by device_id and date_time and aggregating the values separately for float and int columns
df_hourly_float_values = df[all_columns_float_type].groupby(['device_id', pd.Grouper(key='date_time', freq='h')]).agg("mean").reset_index()
df_hourly_int_values = df[all_columns_int_type].groupby(['device_id', pd.Grouper(key='date_time', freq='h')]).agg("max").reset_index()

In [76]:
# merging the two dataframes and outputting the result
merged_df = df_hourly_float_values.merge(df_hourly_int_values, on=['device_id', 'date_time'])
merged_df.to_csv('aggregated_hourly.csv', index=False)